# 🎯 Project Documentation Hub
> Click sidebar **Table of Contents** for navigation

## 📖 Arbasoen - an ahnentafel (pedigree) generator; from GEDCOM to PDF (via LaTeX)

This is *Arbasoen*, a pet project for generating a PDF pedigree file based on a GEDCOM5 structure, using $LaTeX$ as intermediary format.

    Disclaimer: I am not a skilled Python programmer, this is the result
    of an attempt to port a Java based project that I wrote in Java
    approximately 15 years ago to Python in order to learn Python/Colab etc.

### No gdrive mount needed to test

This script offers a few public GEDCOM links which you can use to test, the gedcom will be download to your local storage (notebook storage, no need to mount your gdrive). You can also provide a local path to your own gedcom file on your gdrive which will then be downloaded and processed (the current parameters for the local file point to my personal gedcom file and will not work for you, you need to adapt). This does require you to mount your gdrive obviously.

### Minimal implementation, but functional

The current result is minimal, but functional. The goal is not to deliver a final result, but to provide a base that can be easily extended to your own needs.

* The number of GEDCOM tags that are parsed is currently minimal, adapt to your own needs

* Two languages are foreseen; English and Dutch (Nederlands). The translation is strongly focussed on West-European languages, and the code will probably need some adaptations for different languages. Probably some adaptations are needed for latin languages that make a clear distinction between masculine and feminine words as well


#⚙️ Parameters, this step is needed


In [1]:
#@title Parameter settings

gedcom_examples = {
    "Harry Potter": {
        "url": "https://raw.githubusercontent.com/findmypast/gedcom-samples/refs/heads/main/Harry%20Potter.ged",
        "start_id": "@I00001@",
        "source": "https://github.com/findmypast/gedcom-samples",
        "web_example": True
    },
    "Kennedy": {
        "url": "https://raw.githubusercontent.com/findmypast/gedcom-samples/refs/heads/main/The%20Kennedy%20Family.ged",
        "start_id": "@I0@",
        "source": "https://github.com/findmypast/gedcom-samples",        "web_example": True
    },
    "Game of Thrones": {
        "url": "https://raw.githubusercontent.com/findmypast/gedcom-samples/refs/heads/main/GoT.ged",
        "source": "https://github.com/findmypast/gedcom-samples",
        "start_id": "@I115@",
        "web_example": True
    },
    "Royals 92": {
        "url": "https://raw.githubusercontent.com/arbre-app/public-gedcoms/refs/heads/master/files/royal92.ged",
        "start_id": "@I1@",
        "source": "https://github.com/arbre-app/public-gedcoms",
        "web_example": True
    },

    "Custom Local File": {
        "url": "/content/drive/MyDrive/Arbasoen/gedcom/20241031.ged",
        "start_id": "@I2@",
        "web_example": False
    }
}

#@markdown A few basic examples that you can easily use to test. These run 'as is', no need to mount your drive.

#@markdown Note that you will need to adapt the 'Custom local file' location for it to work with your own local GEDCOM file, in which case you will also need to mount your gdrive.

#@markdown Good examples to select are
#@markdown * Kennedy (small tree, small document)
#@markdown * Royals92 (big tree, big document)
selected_example = "Kennedy"  # @param ["Harry Potter", "Kennedy", "Game of Thrones", "Royals 92", "Custom Local File"]



#@title **Select Your Genealogy Dataset** { run: "auto" }
selected_gedcom_file = gedcom_examples[selected_example]["url"]
start_id = gedcom_examples[selected_example]["start_id"]
web_example = gedcom_examples[selected_example]["web_example"]

#@markdown In case you are building a pedigree for a person, but you do not want to display this person itself (typical case could be: you want to provide additional information like siblings), check this box. The very first generation of the pedigree will be skipped. Make sure to provide some additional $LaTeX$ code to provide the information
# Optional other parameters
skip_first_gen = False  # @param {"type":"boolean"}


#@markdown the resulting language that will be used in the pedigree.
Language = "Nederlands"  # @param ["English", "Nederlands"]
# Map display name to language code
lang_map = {
    "English": "en",
    "Nederlands": "nl"
}

lang = lang_map[Language]


# Resolve final input file based on whether we use web or local file
#if !web_example:
#    arbasoen_path = "/content/drive/MyDrive/Arbasoen/"
#    gedcom_filepath = "gedcom/20241031.ged"
#    selected_gedcom_file = arbasoen_path + gedcom_filepath

print(f"Selected example: {selected_example}")
print(f"\tGEDCOM file: {selected_gedcom_file}")
print(f"\tStart ID: {start_id}")

Selected example: Kennedy
	GEDCOM file: https://raw.githubusercontent.com/findmypast/gedcom-samples/refs/heads/main/The%20Kennedy%20Family.ged
	Start ID: @I0@


#🛠️ Code: GEDCOM Parsing, LaTeX generation classes, lib downloads
This cell (and underlying cells) will mount the gdrive, download the GEDCOM file, either from a remote repo or your local gdrive and process it. It will stop on any fatal error

In [2]:
#@title Pre work. Mount Google Drive (only if using local file), install ged4py and python-dateutil, defines a few auxilary functions


!pip install ged4py # > /dev/null 2>&1
!pip install python-dateutil
!pip install unidecode

if not web_example: # Using local file - gdrive mount is needed
  from google.colab import drive
  drive.flush_and_unmount()
  drive.mount('/content/drive')

from pathlib import Path
import os

fire = " 🔥 "
ok = " ✅ "
nok = " ❌ "

def file_is_readable (file: str) -> bool:
    """
      Check if file exists and is readable.
      Returns True if OK, False otherwise.
    """
    path = Path(file)
    if path.is_file() and os.access(path, os.R_OK):
        return True
    else:
        return False

def read_file(file:str):
    """
      Read the contents of a file
      Args:
        file: path to file
      Returns:
        file contents
    """
    with open(file, 'r') as f:
        return f.read()

def print_first_lines_from_file(file:str, lines:int):
    """
      Prints the first lines from a file, similar to the head function.
      Mostly used for debugging purposes

      Args:
        file: path to file
        lines: number of lines to print
      Returns:
        None - output is printed
    """
    with open(file, 'r') as f:
        for i, line in enumerate(f, 1):
            print(f"Line {i}: {line.rstrip()}")
            if i >= lines:
                break

from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    "Skip cell execution"
    return


#Full gedcom record print
def print_full_record(rec, indent=0):
    """
      Prints a full GEDCOM record, mostly used for debugging purposes
    """
    spacer = "  " * indent
    xref = f" {rec.xref_id}" if rec.xref_id else ""
    val = f" {rec.value!r}" if rec.value is not None else ""
    print(f"{spacer}{rec.level}{xref} {rec.tag}{val}")
    for sub in rec.sub_records:
        print_full_record(sub, indent + 1)


def dir_exists(path: str) -> bool:
    """
      Checks whether the provided directory exists.

      Args:
        path: path to directory
      Return:
        True if OK, False otherwise
    """
    path = Path(path)
    return os.path.isdir(path)

print(ok)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 44.6 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pymeeus: filename=pymeeus-0.5.12-py3-none-any.whl size=732095 sha256=9887f5523e42467e9bb6e9a41af0aeba800f5830776706a969991561e5e2da16
  Stored in directory: /home/codespace/.cache/pip/wheels/92/74/5d/5cf1193a619dc315b5b0a158876900e21440c71130251307b6
Successfully built pymeeus
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [ged4py]2m1/4 [convertdate]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
 ✅ 


In [3]:
#@title Copy gedcom file to /content and validates if the file is readable. Print the first 20 lines of the resulting file for validation
# Use local Colab storage:
gedcom_file = '/content/family.ged'

if (web_example):
  # Download from a URL
  #print("Downloading " + selected_gedcom_file)
  !wget -O "$gedcom_file" "$selected_gedcom_file"
else:
  # Copy local (gdrive) file
  if file_is_readable(gedcom_file):
    print ("Copying " + selected_gedcom_file)
    !cp "$selected_gedcom_file" "$gedcom_file"  # If Drive mounted
  else:
      print("Can't read source file")




# Check GEDCOM file for readability
if file_is_readable(gedcom_file) :
    print(ok + f"Gedcom file (%s) exists and is readable" % gedcom_file)
    print("Displaying the first 20 lines of the file\n\n")
    !head -20 "$gedcom_file"
else:
    # Reset the gedcom_file parameter, so we don't accidentally work with a
    # previous one.
    gedcom_file = ""
    print(nok + f"Missing gedcom_file, or not readable (%s)", gedcom_file)
    raise ValueError("GEDCOM file validation failed - stopping execution. This happens sometimes when fetching remote GEDCOM files, please rerun all cells again")

/content/family.ged: No such file or directory
 ❌ Missing gedcom_file, or not readable (%s) 


ValueError: GEDCOM file validation failed - stopping execution. This happens sometimes when fetching remote GEDCOM files, please rerun all cells again

In [ ]:
#@title Person, Family and Event - base classes for a pedigree

from ged4py import GedcomReader
from dataclasses import dataclass, field
from typing import List, Optional
from datetime import datetime, date
from dateutil.parser import parse as parse_dateutil


@dataclass
class Event:
    """
      Represents a life event (birth, death, marriage, etc.) from GEDCOM data.

      Fields store raw GEDCOM values for flexible parsing later.

      Fields:
        date: Optional[str] = None #raw gedcom date
        place: Optional[str] = None
        type: Optional[str] = None # e.g., "adopted"
    """
    @classmethod
    def from_gedcom(cls,
                    date: Optional[str] = None,
                    place: Optional[str] = None,
                    type_: Optional[str] = None,
                    note: Optional[str] = None):
        """Create Event from GEDCOM fields."""
        event = cls(date=date, place=place, type=type_, note=note)
        return event

    date: Optional[str] = None
    place: Optional[str] = None
    type: Optional[str] = None
    note: Optional[str] = None

@dataclass
class Marriage(Event):
    """
      Marriage-specific event with spouse reference.

      Fields:
        partner_ref: Optional[str] = None  # @I123@ spouse xref
    """
    partner_ref: Optional[str] = None

@dataclass
class Divorce(Event):
    """
      Divorce-specific event with spouse reference.

      Fields:
        partner_ref: Optional[str] = None  # @I123@ spouse xref
    """
    partner_ref: Optional[str] = None

@dataclass
class Person:
    """
      GEDCOM Individual (@Ixxx@) record.

      Core pedigree entity with life events, family links, and computed properties.
    """
    ref: str  # e.g., @I10017@
    name: str = ""
    additional_names: List[str] = field(default_factory=list)

    sex: str = ""  # M/F/U

    # Life events (all default to empty Event)
    birth: Event = field(default_factory=Event)
    baptism: Event = field(default_factory=Event)
    christening: Event = field(default_factory=Event)
    burial: Event = field(default_factory=Event)
    death: Event = field(default_factory=Event)
    christening: Event = field(default_factory=Event)

    # Family structure
    famc_rec: Optional[str] = None
    fams_rec: Optional[str] = None
    fams_recs: List[str] = field(default_factory=list)

    notes: List[str] = field(default_factory=list)

    # Computed (post-processing)
    ahnentafel_ids: List[str] = field(default_factory=list)

    def __str__(self):
        """String representation for printing/debugging."""
        return f"{self.name} ({self.ref}) {self.sex} b.{self.birth.date}"

    def hasParents(self) -> bool:
      """Returns True only for valid, non-empty family refs."""
      famc = self.famc_rec
      return famc is not None and famc.strip() != ""

    def get_famc_rec(self) -> str:
      """Get raw FAMC family reference (or None)."""
      return self.famc_rec

    def died (self) -> bool:
      """True if death event has a date."""
      return self.death.date is not None

    def married(self) -> bool:
        """True if person has spouse families."""
        return len(self.fams_recs) > 0

    def isMale(self) -> bool:
        """True if sex is 'M'."""
        return self.sex == 'M'

    def isFemale(self) -> bool:
        """True if sex is 'F'."""
        return self.sex == 'F'

    def deceased(self) -> bool:
        """True if death event recorded (alias for died())."""
        return self.death.date is not None

    def divorced(self) -> bool:
        """True if divorce event has date (checks generic Event)."""
        return self.divorce.date is not None

@dataclass
class Family:
    """
      GEDCOM Family (@Fxxx@) record - nuclear family unit.

      Links parents (husband/wife) to children.
    """
    ref: str  # e.g., @F7047@
    husband_ref: Optional[str] = None
    wife_ref: Optional[str] = None
    children_refs: List[str] = field(default_factory=list)
    marriage: Event = field(default_factory=Event)
    divorce: Event = field(default_factory=Event)

In [ ]:
#@title Load persons and families (incomplete tag population - to be completed)

from ged4py import GedcomReader
from ged4py.model import Record
import io

persons = {}
families = {}

def extract_event(record, event_tag: str) -> dict:
    """
      Extracts Events (date/place) from BIRT, DEAT, etc.

      Args:
        record: Record
        event_tag: str - e.g., 'BIRT', 'DEAT'
      Returns:
        dict: {'date': str, 'place': str}
    """
    event_rec = record.sub_tag(event_tag)
    if not event_rec:
        return {}
    return {
        'date': event_rec.sub_tag_value('DATE'),
        'place': event_rec.sub_tag_value('PLAC')
    }

def read_person(record: Record) -> Person:
    """
      Read preson information from a GEDCOM file.
      Note that not all GEDCOM tags are currently implemented

      Args:
        record: Record - the GEDCOM person record
      Returns:
        Person: Person object
    """
    person = Person(ref=record.xref_id)

    # Name (first preferred)
    name_recs = record.sub_tags('NAME')
    if name_recs:
        person.name = name_recs[0].value
        person.additional_names = []     # List for variants
        for name_rec in name_recs[1:]:   # All others
            name_value = name_rec.value
            name_type = name_rec.sub_tag('TYPE').value if name_rec.sub_tag('TYPE') else None
            person.additional_names.append({
                'name': name_value,
                'type': name_type  # e.g., 'AKA', 'married', 'nick'
            })
    # Sex
    sex_rec = record.sub_tag('SEX')
    if sex_rec:
        person.sex = sex_rec.value

    # Birth
    birth = extract_event(record, 'BIRT')
    person.birth = Event (birth.get('date'), birth.get('place'))

    # Baptism
    baptism = extract_event(record, 'BAPM')
    person.baptism = Event (baptism.get('date'), baptism.get('place'))

    # Christening
    christening = extract_event(record, 'CHR')
    person.christening = Event (christening.get('date'), christening.get('place'))

    # Death
    death = extract_event(record, 'DEAT')
    person.death = Event (death.get('date'), death.get('place'))

    # Burial
    burial = extract_event(record, 'BURI')
    person.burial = Event (burial.get('date'), burial.get('place'))

    # Parents (FAMC)
    famc_rec = record.sub_tag('FAMC')
    if famc_rec:
        person.famc_rec = famc_rec.xref_id

    # Spouses (FAMS)
    fams_rec = record.sub_tag('FAMS')
    fams_recs = record.sub_tags('FAMS')  # List of FAMS
    person.fams_rec = fams_rec
    person.fams_recs = fams_recs

    # Final result
    return person

def read_family(record: Record) -> Family:
    """
      Read family information from a GEDCOM file.
      Note that not all GEDCOM tags are currently implemented
    """
    husband = record.sub_tag('HUSB')  # Individual or None
    wife = record.sub_tag('WIFE')
    children = record.sub_tags('CHIL')  # List[Individual]

    # Build up marriage information with children
    marriage = extract_event(record, 'MARR')
    divorce = extract_event(record, 'DIV')
    family = Family(
        ref=record.xref_id,
        husband_ref=husband.xref_id if husband else None,
        wife_ref=wife.xref_id if wife else None,
        children_refs=[child.xref_id for child in children],
        marriage = Event (marriage.get('date'), marriage.get('place')),
        divorce = Event (divorce.get('date'), divorce.get('place'))
    )
    return family


with GedcomReader(gedcom_file) as reader:
    """
      Read the gedcom_file and split between individuals (INDI) and Families (FAM). Populate the family structure as global variable afterwards
    """
    # Individuals and Family records
    for record in reader.records0():
      if record.tag == 'INDI':
        person = read_person(record)
        persons[person.ref] = person
      if record.tag == 'FAM':
        family = read_family(record)
        if family:
          families[family.ref] = family
    # Populate the family structure with correct references
    for family in families.values():
      if family.husband_ref:
        husband = persons[family.husband_ref]
        husband.fams_recs.append(family.ref)
      if family.wife_ref:
        wife = persons[family.wife_ref]
        wife.fams_recs.append(family.ref)


print(f"Loaded {len(persons)} persons")
print(f"Loaded {len(families)} families")

Loaded 70 persons
Loaded 19 families


In [ ]:
#@title Building the generation structure
for p in persons.values():
      p.ahnentafel_ids = []  # Clear

root = persons[start_id]

from collections import deque, defaultdict
from typing import Dict, Set, List
from dataclasses import dataclass
from typing import Dict, List, Tuple

import pdb

@dataclass
class PathCtx:
    person: 'Person'
    ahnen_id: int
    gen: int

generations = defaultdict(list)
ahnen_map = {}
path_visited = set()

# Root
root.ahnentafel_ids = [1]  # Root only
queue = deque([PathCtx(root, 1, 0)])
ahnen_map[1] = root
generations[0].append((1, root))
path_visited.add((root.ref, 1))

def add_unique_id(person: 'Person', ahnen_id: int):
    if ahnen_id not in person.ahnentafel_ids:
        person.ahnentafel_ids.append(ahnen_id)

def build_ancestor_generations(persons: Dict[str, 'Person'], families: Dict[str, 'Family']) -> Dict[int, List[Tuple[int, 'Person']]]:
    """
      The main function. Takes the root person, places the parents of this
      person in the queue and processes the queue while there are person
      records in it

      Args:
        persons: Dict[str, Person] - dict of all persons
        families: Dict[str, Family] - dict of all families
      Returns:
        Dict[int, List[Tuple[int, Person]]] - dict of generations
    """
    visited: Set[str] = set()
    path_visited = set()
    MAX_GEN = 23

    # As long as we have persons in our queue
    while queue:
      ctx = queue.popleft()

      if ctx.gen >= MAX_GEN:
        continue

      # Fresh parents from THIS path only
      if ctx.person.famc_rec and ctx.person.famc_rec in families:
        family = families[ctx.person.famc_rec]

        # Father
        if family.husband_ref and family.husband_ref in persons:
            father = persons[family.husband_ref]
            father_id = ctx.ahnen_id * 2
            if father_id not in ahnen_map:
                ahnen_map[father_id] = father
                generations[ctx.gen + 1].append((father_id, father))
                path_key = (father.ref, father_id)
                if path_key not in path_visited:
                    path_visited.add(path_key)
                    queue.append(PathCtx(father, father_id, ctx.gen + 1))
        # Mother.
        if family.wife_ref and family.wife_ref in persons:
            mother = persons[family.wife_ref]
            mother_id = ctx.ahnen_id * 2 + 1
            if mother_id not in ahnen_map:
                ahnen_map[mother_id] = mother
                generations[ctx.gen + 1].append((mother_id, mother))
                path_key = (mother.ref, mother_id)
                if path_key not in path_visited:
                    path_visited.add(path_key)
                    queue.append(PathCtx(mother, mother_id, ctx.gen + 1))

    # After loop: sort each generation by Ahnentafel ID
    for gen_list in generations.values():
      gen_list.sort(key=lambda x: x[0])

    # Backfill ahnentafel_ids on persons
    for person in persons.values():
      ids = [aid for aid, p in ahnen_map.items() if p.ref == person.ref]
      person.ahnentafel_ids = ids[:10]

    return dict(generations)


generations = build_ancestor_generations(persons, families)
for person in persons.values():
    person.ahnentafel_ids = [id for id, p in ahnen_map.items() if p.ref == person.ref]


In [ ]:
#@title Some pedigree related functions

import re

newline="\n"

def escape_latex_underscore(name: str) -> str:
    """
      Escape underscores for LaTeX by replacing '_' with '\\_'.

      Args:
        name: str - the name to escape
      Returns:
        str: the escaped name
    """
    return str(name).replace('_', r'\_')


def ahnentafel_generation(ahnentafel_id: int) -> int:
    """
      Generation where ID 1=gen 1, 2-3=gen 2, 4-7=gen 3, etc.
      Args:
        ahnentafel_id: int
      Returns:
        int: the generation based on the ahnentafel_id
    """
    if ahnentafel_id < 1:
        raise ValueError("Ahnentafel ID must be >= 1")
    return math.floor(math.log2(ahnentafel_id)) + 1


def get_father_children(father: Person, families: Dict[str, Family]) -> List[str]:
    """
      Return child refs for given father
      Args:
        father: Person
        families: Dict[str, Family]
      Returns:
        List[str]: list of child refs
    """
    children = []
    for family in families.values():
        if family.husband_ref == father.ref:
             children.extend(family.children_refs)
    return children


def get_family_children(famc_rec, families: Dict[str, Family]) -> List[str]:
    """
      Return child refs for given father
      Args:
        famc_rec (str): the family record ref
        families (Dict[str, Family]): dict of all known families
      Returns:
        List[str]: list of child refs
    """
    children = []
    for family in families.values():
        if family.ref == famc_rec:
             children.extend(family.children_refs)
    return children


def pretty_name(gedcom_name) -> str:
    """
      Formats a GEDCOM name OR plain name into a human-readable string.
      Args:
        gedcom_name: str or tuple
      Returns:
        str: the formatted name (LaTeX-escaped)
      Example:
        >>> pretty_name("Barry /Nauta/")
        'Barry Nauta'
        >>> pretty_name("Barry Nauta")
        'Barry Nauta'
        >>> pretty_name("Barry_Nauta")
        'Barry\\_Nauta'
    """
    # Handle tuple (given, surname) OR string
    if isinstance(gedcom_name, (tuple, list)):
        parts = [escape_latex_underscore(p.strip()) for p in gedcom_name if p]
        return ' '.join(parts)

    # String fallback: GEDCOM "John /Doe/" → "John Doe" OR plain "John Doe"
    gedcom_name = str(gedcom_name).strip()
    match = re.match(r'^(.+?)\s*/([^/]+?)/\s*(.*)?$', gedcom_name)
    if match:
        given, surname, suffix = match.groups()
        parts = [escape_latex_underscore(given.strip()), escape_latex_underscore(surname.strip())]
        if suffix:
            parts.append(escape_latex_underscore(suffix.strip()))
        return ' '.join(parts)

    # Plain name fallback (handles "Barry Nauta", "Barry_Nauta", etc.)
    return escape_latex_underscore(gedcom_name)


def pretty_place(gedcom_place, part=1) -> str:
    """
      Formats a GEDCOM place into a human-readable string.

      Args:
        Gedcom_place: str or tuple
        part: int (1-indexed) - the part of the place to return
      Returns:
        str: the indexed place-part of the input
      Example:
        >>> pretty_place("New York, USA")
        'New York'
    """
    if not gedcom_place:
        return ""
    parts = [p.strip() for p in gedcom_place.split(',')]
    if 1 <= part <= len(parts):
        return parts[part - 1]  # 1-indexed
    return ""


def surname_first(name_input) -> str:
    """
      Convert GEDCOM name OR plain name to "Surname, Given" format.

      Args:
        name_input: str or tuple
      Returns:
        str: "Surname, Given" (LaTeX-escaped)
      Example:
        >>> surname_first("Barry /Nauta/")
        'Nauta, Barry'
        >>> surname_first("Barry Nauta")
        'Barry Nauta'
    """
    # Convert tuple to string if needed
    if isinstance(name_input, tuple):
        if len(name_input) >= 2 and name_input[1]:
            return f"{escape_latex_underscore(name_input[1])}, {escape_latex_underscore(name_input[0])}"
        return escape_latex_underscore(str(name_input[0])) if name_input else ""

    # Original string parsing (for raw GEDCOM)
    gedcom_name = str(name_input).strip()
    if not gedcom_name:
        return ''

    parts = re.split(r'/', gedcom_name, maxsplit=2)

    if len(parts) == 3:
        given = parts[0].strip()
        surname = parts[1].strip()
        suffix = parts[2].strip()
        given_parts = [escape_latex_underscore(p) for p in [given, suffix] if p]
        return f"{escape_latex_underscore(surname)}, {' '.join(given_parts)}"
    elif len(parts) == 2 and parts[1].strip():  # Fixed: parts[1] is surname
        return f"{escape_latex_underscore(parts[1].strip())}, {escape_latex_underscore(parts[0].strip())}"

    # Plain name fallback
    return escape_latex_underscore(gedcom_name)


def get_firstname(name_input) -> str:
    """
      Extracts ONLY firstname from GEDCOM name tuple/string/Person.
      GEDCOM tuple format: (given1, surname, given2/suffix)

      Args:
        name_input: str or tuple
      Returns:
        str: the firstname (LaTeX-escaped)
      Example:
        >>> get_firstname("Barry /Nauta/")
        'Barry'
        >>> get_firstname("Barry Nauta")
        'Barry'
    """
    if hasattr(name_input, 'name'):
        name_data = name_input.name
    else:
        name_data = name_input

    # Handle tuple: take ONLY index 0 (given1/firstname)
    if isinstance(name_data, tuple) and len(name_data) > 0:
        firstname = name_data[0].strip()
        return escape_latex_underscore(firstname.split()[0]) if firstname else ""

    # Handle string (fallback)
    gedcom_name = str(name_data).strip()
    if not gedcom_name or '/' not in gedcom_name:
        # Plain name: take first word
        return escape_latex_underscore(gedcom_name.split()[0]) if gedcom_name.split() else ""

    firstname = gedcom_name.split('/', 1)[0].strip()
    return escape_latex_underscore(firstname.split()[0]) if firstname else ""


def calculate_generation(ahnentafel_id: int) -> int:
    """
      Ahnentafel generation: floor(log2(ID))
      Args:
        ahnentafel_id: int
      Returns:
        int: the generation based on the ahnentafel_id
      Example:
        >>> calculate_generation(1)
        0
        >>> calculate_generation(2)
        1
    """
    return math.floor(math.log2(ahnentafel_id)) if ahnentafel_id > 0 else 0


In [ ]:
#@title LaTeX helper functions

import unicodedata
import re
from unidecode import unidecode

class LaTeX:

    def math(input: str) -> str:
      """
        Return a math version of the provided text
        Args:
          input: the input string
        Result:
          str: the math version of the input (wrapped)
      """
      return f"$^{{{input}}}$"


    def href(url: str, text: str) -> str:
      """
        Creates a LaTeX hypertarget command, this is a link internal to a an external link

        Args:
          url: str - target name, the reference
          text: str - target text, the text to display
        Returns:
          str: LaTeX href
      """
      return f"\\href{{{url}}}{{{text}}}"


    def hyperlink(name: str, text: str) -> str:
      """
        Creates a LaTeX hypertarget command, a link to an internal anchor

        Args:
          name: str - target name, the reference
          text: str - target text, the text to display
        Returns:
          str: LaTeX hypertarget
      """
      return f"\\protect\\hyperlink{{{name}}}{{{text}}}"


    def hypertarget(name: str, text: str) -> str:
      """
        Creates a LaTeX hypertarget command, this is the internal anchor

        Args:
          name: str - target name, the reference
          text: str - target text, the text to display
        Returns:
          str: LaTeX hypertarget
      """
      return f"\\hypertarget{{{name}}}{{{text}}}"


    def itemize(input: List[str]) -> str:
      """
        Creates a LaTeX itemize command based on the list input

        Args:
          input: List of items to itemise
        Returns:
          str: LaTeX itemize command with items
      """
      result = "\\begin{itemize}\n"
      for item in input:
        result += "\t\\item " + item + "\n"
      result += "\\end{itemize}\n\n"
      return result


    def section(input: str, star: bool = False) -> str:
      """
        Creates a LaTeX section command with optional star (default False)
      """
      star_part = "*" if star else ""
      result = "\\section" + star_part + f"{{{input}}}"
      return result


    def subsection(input: str, star: bool = False) -> str:
      """
        Creates a LaTeX subsection command with optional star (default False)
      """
      star_part = "*" if star else ""
      result = "\\subsection" + star_part + f"{{{input}}}"
      return result


    def subsubsection(input: str, star: bool = False) -> str:
      """
        Creates a LaTeX subsubsection command with optional star (default False)
      """
      star_part = "*" if star else ""
      result = "\\subsubsection" + star_part + f"{{{input}}}"
      return result


    def index_key(index_key: str, input: List['str']) -> str:
      """
        Generate LaTeX index command from key and list of subentries.
      """
      parts = [index_key] + input
      formatted = '!'.join(parts)
      return f"\\index{{{formatted}}}\n"


    def wrapfigure(height: int, width: int, image_location: str) -> str:
      """
        LaTeX wrapfigure function. Currently not used in this project
      """
      result = ""
      if image_location:
        result += f"\\begin{{wrapfigure}}{{l}}[{height}pt]{{{width}pt}}[10pt]\n"
        result += f"\t\\includegraphics[width=0.9\\width]{{{image_location}}}\n"
        result += "\\end{wrapfigure}\n"
      return result


    def slugify(filename):
      """
        Remove all unwanted characters from the string input
        (typically used for filenames, as the method name suggests)
        Not necessarily a LaTeX command, candidate to move elsewhere
      """
      # Step 1: Deaccent (ñ→n, ç→c, ä→a)
      name = unidecode(filename)
      # Step 2: Remove/replace specials (underscores → nothing or dash)
      name = re.sub(r'[_]+', '', name)  # Remove _
      name = re.sub(r'[^\w\s.-]', '', name)  # Alphanum + spaces/dots/dash
      name = re.sub(r'[\s]+', '_', name)  # Spaces → single _
      return name.strip('_.-')

In [ ]:
#@title Translations

class Translations:
    def __init__(self, lang='nl'):
        self.lang = lang
        self._data = {
            'en': {
                'months': {
                    'JAN': 'January', 'FEB': 'February', 'MAR': 'March',
                    'APR': 'April', 'MAY': 'May', 'JUN': 'June',
                    'JUL': 'July', 'AUG': 'August', 'SEP': 'September',
                    'OCT': 'October', 'NOV': 'November', 'DEC': 'December'
                },
                'between': 'between',
                'after': 'after',
                'and': 'and',
                'from': 'from',
                'from_range': 'from',
                'to': 'to',
                'about': 'about',
                'before': 'before',
                'around': 'around',
                'of': 'of',
                'on': 'on',
                'calculated': 'calculated',
                'estimated': 'estimated',
                'interpreted': 'interpreted as',
                'married': 'married',
                'with': 'with',
                'ongoing': 'ongoing',
                'Generation': 'Generation',
                'See': 'See',
                'Pedigree': 'Pedigree',
                'their_children': 'They had the following children:',
                'and_is_burried': ' and is buried',
                'died': 'died',
                'unknown': 'unknown',
                'son': 'son',
                'daughter': 'daughter',
                'husband': 'husband',
                'wife': 'wife',
                'child': 'child',
                'person_by_familyname': 'Person by family name',
                'person_by_firstname': 'Person by first name',
                'born': 'born',
                'burial': 'burial',
                'buried': 'buried',
                'baptism': 'baptism',
                'christening': 'christening',
                'death': 'death',
                'event_by_place': 'Event by place',
                'are_divorced_on': 'are divorced on',
                'are_married_on': 'are married on',
                'is_baptised_on': 'is baptised on',
                'is_christened_on': 'is christened on',
                'is_born': 'is born'
                # Add more terms as needed
            },
            'nl': {
                'months': {
                    'JAN': 'januari', 'FEB': 'februari', 'MAR': 'maart',
                    'APR': 'april', 'MAY': 'mei', 'JUN': 'juni',
                    'JUL': 'juli', 'AUG': 'augustus', 'SEP': 'september',
                    'OCT': 'oktober', 'NOV': 'november', 'DEC': 'december'
                },
                'between': 'tussen',
                'after': 'na',
                'and': 'en',
                'from': 'van',
                'from_range': 'vanaf',
                'to': 'tot',
                'about': 'rond',
                'before': 'voor',
                'around': 'rond',
                'of': 'van',
                'on': 'op',
                'calculated': 'berekend',
                'estimated': 'geschat',
                'interpreted': 'geinterpreteerd als',
                'married': 'trouwde',
                'with': 'met',
                'ongoing': 'ongoing',
                'Generation': 'Generatie',
                'See': 'Zie',
                'Pedigree': 'Stamboom',
                'their_children': 'Ze hadden de volgende kinderen:',
                'and_is_burried': ' en is begraven',
                'died': 'stierf',
                'unknown': 'onbekend',
                'son': 'zoon',
                'daughter': 'dochter',
                'husband': 'man',
                'wife': 'vrouw',
                'child': 'kind',
                'person_by_familyname': 'Persoon per familienaam',
                'person_by_firstname': 'Persoon per voornaam',
                'born': 'geboren',
                'burial': 'begraven',
                'buried': 'begraven',
                'baptism': 'doop',
                'christening': 'doop',
                'death': 'overlijden',
                'event_by_place': 'Gebeurtenis by plaats',
                'are_divorced_on': 'zijn gescheiden op',
                'are_married_on': 'zijn getrouwd op',
                'is_baptised_on': 'is gedoopt op',
                'is_christened_on': 'is gedoopt op',
                'is_born': 'is geboren'
                # Add more terms as needed
            }
        }

    def t(self, key, category=None):
        """Translate key. If category provided, looks in category dict."""
        if category:
            return self._data.get(self.lang, {}).get(category, {}).get(key, key)
        return self._data.get(self.lang, {}).get(key, key)

    def set_lang(self, lang):
        self.lang = lang

In [ ]:
#@title DateVisitor and date helper functions
from ged4py.date import DateValue, DateValueVisitor
from ged4py.date import DateValueTypes as DateTypes
import pdb


tr = Translations(lang)
months = tr.t('months')
qualifiers = ['ABOUT ', 'ABT ', 'ABT.', 'BEF ', 'BEF.', 'BEFORE ', 'VOR ', 'AFT ', 'AFT.', 'AFTER ', 'BET ', 'BETWEEN ', 'FROM ', 'TO ']
translated_quals = {
        'ABOUT ': tr.t('about'), 'ABT ': tr.t('about'),
        'BEF ': tr.t('before'), 'VOR ': tr.t('before'), 'BEFORE ': tr.t('before'),
        'AFT ': tr.t('after'), 'AFTER ': tr.t('after'),
        'BET ': tr.t('between'), 'BETWEEN ': tr.t('between'),
        'FROM ': tr.t('from_range'), 'TO ': tr.t('to')
    }

def full_month(date_str: str) -> str:
    """
      Convert GEDCOM date to full month names.
    """
    date_value = DateValue.parse(date_str)
    if date_value.kind == DateTypes.SIMPLE and date_value.date.month:
        #full_mon = GED_MONTHS.get(date_value.date.month, date_value.date.month)
        month_key = date_value.date.month  # e.g. 'JAN', 'FEB'
        full_mon = tr.t(month_key, category='months')
        return f"{date_value.date.day} {full_mon} {date_value.date.year}" if date_value.date.day and date_value.date.year else str(date_value)
    return str(date_value)  # Fallback

class PrettyDateVisitor(DateValueVisitor):
    """
      Visitor to generate human-readable 'pretty' strings for GEDCOM dates.
    """
    def visitSimple(self, date):
        if date.date and date.date.month:
            full_mon = GED_MONTHS[date.date.month]
            return f"{date.date.day or ''} {full_mon} {date.date.year or ''}".strip()
        return str(date.date) or ''

    def visitPeriod(self, date):
        return f"{tr.t('from')} {date.date1} {tr.t('to')} {date.date2}"

    def visitFrom(self, date):
        return f"{tr.t('from_range')} {date.date} ({tr.t('ongoing')})"

    def visitTo(self, date):
        return f"{tr.t('to')} {date.date}"

    def visitRange(self, date):
        return f"{tr.t('between')} {date.date1} {tr.t('and')} {date.date2}"

    def visitBefore(self, date):
        return f"{tr.t('before')} {date.date}"

    def visitAfter(self, date):
        return f"{tr.t('after')} {date.date}"

    def visitAbout(self, date):
        return f"{tr.t('around')} {date.date}"

    def visitCalculated(self, date):
        return f"{tr.t('calculated')} {date.date}"

    def visitEstimated(self, date):
        return f"{tr.t('estimated')} {date.date}"

    def visitInterpreted(self, date):
        return f"{tr.t('interpreted')} {date.date} ({date.phrase})"

    def visitPhrase(self, date):
        if hasattr(date, 'date') and date.date:  # Qualified phrase like "ABOUT <date>"
          inner = self.visitSimple(date)  # Reuse simple formatter
          return f"{tr.t('around')} {inner}"
        return date.phrase or str(date)

    def generic_visit(self, date):
        return str(date)


    def visitPhrase(self, date):
        phrase = getattr(date, 'phrase', '').strip()
        if not phrase:
            return str(date).strip()

        # Strip parens if present
        phrase = phrase.strip('() ')

        # Qualified? Check if parseable inner date dd/MM/yyy
        if '/' in phrase and re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', phrase):
            try:
                dt = datetime.strptime(phrase, '%d/%m/%Y')
                full_mon = GED_MONTHS.get(dt.strftime('%b').upper(), dt.strftime('%B'))
                return f"{dt.day} {full_mon} {dt.year}"
            except ValueError:
                pass

        return phrase


    def visitDateValueSimple(self, date):
        return self.visitSimple(date)

def pretty_date(date_input) -> str:
    date_str = str(date_input) if not isinstance(date_input, str) else date_input
    """
      Parse a GEDCOM date string and return a pretty human-readable version.

      Handles qualifiers like 'before', 'after', 'around/between', 'from/to', phrases, etc.

      Example:
        >>> pretty_date("BEF 1950")  # 'before 1950'
        >>> pretty_date("BET 1920 AND 1930")  # 'between 1920 and 1930'
        >>> pretty_date("ABT 1800")  # 'around 1800'
    """
    if not date_str:
        return ""

    date_str = str(date_input).strip() if date_input else ""
    if not date_str:
        return ""

    # Manual qualifiers (recurse safe)
    # BEFORE and VOR are hacks
    qualifiers = ['ABOUT ', 'ABT ', 'ABT.', 'BEF ', 'BEF.', 'BEFORE ', 'VOR ', 'AFT ', 'AFT.', 'AFTER ', 'BET ', 'BETWEEN ', 'FROM ', 'TO ']
    for qual in qualifiers:
        if date_str.upper().startswith(qual.upper()):
            inner = date_str[len(qual):].strip()
            # Fix AND before recurse
            inner = inner.replace('AND', 'en').replace('and', 'en')
            inner_pretty = pretty_date(inner) or inner
            return translated_quals.get(qual.upper(), qual) + ' ' + inner_pretty
    # Month replace
    for abbr, full in months.items():
        date_str = date_str.replace(abbr, full.capitalize())


    try:
        from ged4py.date import DateValue, DateValueVisitor
        date_value = DateValue.parse(date_str)
        visitor = PrettyDateVisitor()
        result = date_value.accept(visitor) or date_str
        return result
    except ImportError:
        pass
    except Exception as e:
        pass
    return date_str  # Always str

def is_approximate(date_input: str) -> bool:
    """
      Returns whether a date is an approximation (like 'before 1900', 'around 1900', 'after 1900' etc)

      Args:
        date_input: str - the date to check
      Returns:
        bool
    """
    date_str = str(date_input or "").strip()
    if not date_str:
        return False  # Exact (empty)
    qualifiers = ['ABOUT ', 'ABT ', 'ABT.', 'BEF ', 'BEF.', 'BEFORE ', 'VOR ', 'AFT ', 'AFT.', 'AFTER ', 'BET ', 'BETWEEN ', 'FROM ', 'TO ']
    return date_str.upper().startswith(tuple(q.upper() for q in qualifiers))


def is_gedcom_month_year_only(date_str):
    """
      Returns whether a date is built up of month and year only (not containing the day)

      Args:
        date_str: str - the date to check
      Returns:
        bool
    """
    if not date_str or not isinstance(date_str, str):
        return False

    # Strip prefixes: qualifiers, calendars (case-insensitive)
    cleaned = re.sub(r'^\s*(?:@#D[^@]+@|ABT|AFT|BEF|BET|FROM|TO|CAL|EST)\s*', '', date_str.strip(), flags=re.IGNORECASE)

    parts = cleaned.split()

    if len(parts) == 2:
        month, year_str = parts
        month_upper = month.upper()
        months = {'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'}
        try:
            year = int(year_str)
            return (month_upper in months and abs(year) >= 100)  # Reasonable year check
        except ValueError:
            pass
    return False


In [ ]:
#@title LaTeXGenerationVisitor - visitor class to construct the LaTeX document
from abc import ABC, abstractmethod
from typing import List, Dict, Any, Protocol, Tuple
from dataclasses import dataclass
import math
import re
import sys
from io import StringIO
from contextlib import redirect_stdout
from typing import List

newline="\n"


class GenerationVisitor(ABC):
    """Abstract base for per-generation LaTeX"""
    @abstractmethod
    def visit_generation(gen_num: int, generation: List[Tuple[int, 'Person']], families: Dict[str, Family]):
        pass

class LatexGenerationVisitor(GenerationVisitor):

    def comments_for_person(self, person: 'Person', ahnen_id: int) -> str:
        """
          Provide a LaTeX comment section for a person - for easy navigation
          in the LaTeX file
        """
        comments = (f"%----------------------------------------------------\n" +
                    f"%  {pretty_name(person.name)}                        \n" +
                    f"%  {person.ref}                                      \n" +
                    f"%----------------------------------------------------\n" +
                    f"{LaTeX.hypertarget(ahnen_id, "")}\n")
        return comments


    def visit_person_id_and_title(self, ahnen_id: int, person: 'Person') -> str:
        """
          Provides a latex subsection for a person with the ahnen_id and name
          as title
        """
        sectionTitle = str(ahnen_id) + ". " + pretty_name(person.name)
        return LaTeX.subsubsection(sectionTitle)


    def visit_person_create_indices(self, ahnen_id: int, person: 'Person') -> str:
        """
          Create indices for a person, based on names and some life events
        """
        gen = calculate_generation(ahnen_id) + 1
        gen_filled = str(gen).zfill(3)
        ahnen_filled = str(ahnen_id).zfill(10)
        name_str = pretty_name(person.name)
        surname_first_str = surname_first(person.name)
        display_gen = f"{tr.t('Generation')} {gen}"
        sort_entry = f"{ahnen_filled}@{ahnen_id}. {name_str}"

        key = f"{tr.t('Generation')} {gen_filled}@\\textbf{{{tr.t('Generation')} {gen}}}"
        indices = LaTeX.index_key(key,
          [tr.t('Generation') + " " + str(gen), ahnen_filled + '@' + str(ahnen_id) + '. ' + name_str] )
        indices += LaTeX.index_key("1@\\textbf{" + tr.t("person_by_firstname") + "}", [name_str])
        indices += LaTeX.index_key("2@\\textbf{" + tr.t("person_by_familyname") + "}", [surname_first_str])

        #
        # Todo check for approx birth and fix bap/chr
        #
        # Idem for death/burial
        #

        indices += self.index_event_place(person, person.birth, "birth")
        indices += self.index_event_place(person, person.baptism, "baptism")
        indices += self.index_event_place(person, person.christening, "christening")
        indices += self.index_event_place(person, person.death, "death")
        indices += self.index_event_place(person, person.burial, "burial")
        #indices += self.index_event_place(person, person.xxx, "marriage")
        #indices += self.index_event_place(person, person.xxx, "divorce")
        return indices


    def index_event_place(self, person: 'Person', event, event_name: str) -> str:
        """
          Creates an place-index for an event for a person
        """
        event_str = []
        index = ""
        if event.place:
          event_str.append(pretty_place(event.place))
          name = pretty_name(person.name)
          if event.date:
            name += f" {event_name} {pretty_date(event.date)}"
            event_str.append(name)
          index = LaTeX.index_key("3@\\textbf{"+tr.t("event_by_place")+"}", event_str)
        return index


    def link_person_to_parents(self, ahnen_id: int, person: 'Person') -> str:
        result = ""
        """
          Provide links to the parents (if known)
        """
        # Only print if any parent is known
        if (person.hasParents()):
            assert person.hasParents()
            result += pretty_name (person.name) + ", "
            if person.isMale():
              result += tr.t('son')
            elif person.isFemale():
              result += tr.t('daughter')
            else:
                result += tr.t('child')
            result += " " + tr.t('of') + " "
            # Get family ref and object
            famc = person.famc_rec
            if famc and famc in families:
                family = families[famc]
                # Father information
                if family.husband_ref and family.husband_ref in persons:
                    father = persons[family.husband_ref]
                    result += pretty_name(father.name)
                    result += " (" + LaTeX.math (LaTeX.hyperlink(ahnen_id*2, ahnen_id*2)) + ")"
                else:
                    result += " " + tr.t('unknown')
                result += " "+ tr.t('and') + " "
                # Mother information
                if family.wife_ref and family.wife_ref in persons:
                    mother = persons[family.wife_ref]
                    result += pretty_name(mother.name)
                    result += " (" + LaTeX.math (LaTeX.hyperlink(ahnen_id*2+1, ahnen_id*2+1)) + ")"
                else:
                    result += " " + tr.t('unknown')
            result += "."
        return result


    def marriage_info(self, person: 'Person', recurse_into_children: bool = True) -> str:
        """
          Provides marriage information for a person, with optional information
          for children (this is the case for a main person, but not when we are printing information when this person is listed as a child)
        """
        result = ""
        if (person.fams_recs is not None):
          for fams_rec in person.fams_recs:
              xref = fams_rec.value if hasattr(fams_rec, 'value') else fams_rec #else fams_rec.lstrip('@').rstrip('@')
              spouse = None
              if xref is not None and xref in families:
                family = families[xref]
                if family.wife_ref and family.wife_ref != person.ref and family.wife_ref in persons:
                  spouse = persons[family.wife_ref]
                elif family.husband_ref and family.husband_ref != person.ref and family.husband_ref in persons:
                  spouse = persons[family.husband_ref]
                result += get_firstname(person.name) + " " + tr.t('married')

                if family.marriage.date is not None:
                  if not is_approximate(family.marriage.date):
                    result += " " + tr.t('on') + " "
                  result += pretty_date(family.marriage.date)
                if spouse and spouse.name is not None:
                  spouse_str = pretty_name(spouse.name)
                  if spouse_str is not None:
                    result += " " + tr.t('with') + " " + pretty_name(spouse.name)
                  if len(spouse.ahnentafel_ids) > 1:
                    result += " (" + LaTeX.math(LaTeX.hyperlink(spouse.ahnentafel_ids[0], spouse.ahnentafel_ids[0])) + ")"
                if family.marriage.place is not None:
                  result += " "+ tr.t('in') + " " + pretty_place(family.marriage.place)
                result += ". "
                if family.divorce.date is not None:
                  result += get_firstname(person.name) + " en " + get_firstname(spouse.name)
                  result += " " + tr.t("are_divorced_on") + " " + pretty_date(family.divorce.date) + "."
                # Todo - list the children of a woman if not listed at spouse
                # (i.e. second marriage)
                if recurse_into_children and person.isMale(): #or spouse not in ahnentafel_map?
                  result += self.visit_children_of_person(person, family.children_refs)
        return result



    def birth_info(self, person: 'Person', full_version: bool=False) -> str:
        """
          Display birth information. Birth as well as baptism/christening.
        """

        minimum_info_provided = False
        result = ""

        if ((person.birth is not None or is_approximate(person.birth.date))
                and not (person.baptism.date is None and person.christening.date is None)
                and not
                (is_approximate(person.baptism.date)) or (is_approximate(person.christening.date))
            ):
            result = get_firstname(person.name)
            result += " " + tr.t("is_baptised_on") + " "
            if person.baptism.date is not None:
                result += pretty_date(str(person.baptism.date))
                if person.baptism.place is not None:
                    result += " "+ tr.t('in') + " " + pretty_place(person.baptism.place)
            else:
                result += pretty_date(str(person.christening.date))
                if person.christening.place is not None:
                    result += " "+ tr.t('in') + " " + pretty_place(person.christening.place)
            result += ". "
            return result
        if person.birth is not None and person.birth.date is not None:
            result = get_firstname(person.name)
            if (person.birth.date is not None or person.birth.place is not None):
                result += " " + tr.t("is_born") + " "
            if (person.birth.date is not None):
              if not is_approximate(person.birth.date):
                if is_gedcom_month_year_only(person.birth.date):
                  result += tr.t('in') + " "
                else:
                  result += tr.t('on') + " "
              result += pretty_date(str(person.birth.date))
            if person.birth.place is not None:
                result += " "+ tr.t('in') + " " + pretty_place(person.birth.place)
            minimum_info_provided = True
        if full_version or not minimum_info_provided:
          if person.baptism.date is not None or person.christening.date is not None:
            result += " " + tr.t("and") + " " + tr.t("is_baptised_on") + " "
            if person.baptism.date is not None:
                result += pretty_date(str(person.baptism.date))
                if person.baptism.place is not None:
                    result += " "+ tr.t('in') + " " + pretty_place(person.baptism.place)
            else:
                result += pretty_date(str(person.christening.date))
                if person.christening.place is not None:
                    result += " "+ tr.t('in') + " " + pretty_place(person.christening.place)
        if result == "":
          return result
        result += ". "
        return result



    def death_info(self, person: 'Person') -> str:
        """
          Display death information. Death as well as burial.
        """
        result = get_firstname(person.name)
        if ((is_approximate(person.death.date)) and not
                (person.burial.date is None or is_approximate(person.burial.date))
            ):
          result += " " + tr.t("is_buried_on") + " "
          if person.burial.date is not None:
                result += pretty_date(str(person.burial.date))
                if person.burial.place is not None:
                    result += " "+ tr.t('in') + " " + pretty_place(person.burial.place)
          else:
                result += pretty_date(str(person.christening.date))
                if person.christening.place is not None:
                    result += " "+ tr.t('in') + " " + pretty_place(person.christening.place)
          result += ". "
          return result

        if person.death is not None and person.death.date is not None:
            result += " "+ tr.t('died') + " "
            if not is_approximate(person.death.date):
              if is_gedcom_month_year_only(person.death.date):
                result += tr.t('in') + " "
              else:
                result += tr.t('on') + " "
            result += pretty_date(str(person.death.date))
            if person.death.place is not None:
                result += " "+ tr.t('in') + " " + pretty_place(person.death.place)
        if person.burial is not None and person.burial.date is not None:
            result += " " + tr.t('and_is_burried') + " "
            if not is_approximate(person.burial.date):
              if is_gedcom_month_year_only(person.burial.date):
                result += tr.t('in') + " "
              else:
                result += tr.t('on') + " "
            result += pretty_date(str(person.burial.date))
            if person.burial.place is not None:
                result += " "+ tr.t('in') + " " +  pretty_place(person.burial.place)
        result +=". "
        return result


    def print_life_stats(self, ahnen_id: int, person: 'Person', persons: Dict[str, 'Person']) -> str:
        """
          Print life stats for a person. Birth, marriage (incl divorce) and
          death
        """
        result = self.birth_info(person, True)
        if person.married():
            result += self.marriage_info (person)
        if person.died():
            result += self.death_info (person)
        result += "\n"
        return result


    def child_info(self, person: 'Person', ahnen_id: int, persons: Dict[str, 'Person']) -> str:
      """
        Print child information for a person, including a link to a person,
        if this person is known in the ahnen_tafel
      """
      info = pretty_name(person.name)
      if len(person.ahnentafel_ids) > 1:
        info += " ("+LaTeX.math(LaTeX.hyperlink(person.ahnentafel_ids[0], person.ahnentafel_ids[0])) + ")"
      info += ". "

      if person.birth.date or person.baptism.date or person.christening.date:
        info += self.birth_info (person, False)
      if person.married():
        info += self.marriage_info (person, False)
      if person.died():
        info += self.death_info (person)
      return info


    def visit_children_of_person(self, person: 'Person', children: List[Person]) -> str:
        """
          Print a list of children (with their main events,
          but skip their children) for this person
        """
        result = ""
        names_list=[]
        if (len(children) > 0):
            result += tr.t('their_children') + " \n"
            for child_id in children:
                child = persons[child_id]
                names_list.append(self.child_info(child, child_id, persons))
            result += LaTeX.itemize(names_list)
        return result



    def visit_person(self, ahnen_id: int, person: 'Person') -> str:
        """
          Visit a person and print the main information.
          his includes the main comments in the LaTeX file, indices, link to
          parents as well as children
        """
        if (len(person.ahnentafel_ids) > 1) and (person.ahnentafel_ids[0] != ahnen_id):
            result = newline + self.visit_person_id_and_title(ahnen_id, person)
            result += newline + self.comments_for_person(person, ahnen_id)
            result += tr.t('See') + " " + LaTeX.math(LaTeX.hyperlink(person.ahnentafel_ids[0], person.ahnentafel_ids[0]))
            return result
        person.ahnentafel_ids.append(ahnen_id)
        result = (newline + self.comments_for_person(person, ahnen_id) +
                  newline + self.visit_person_id_and_title(ahnen_id, person) +
                  newline + self.visit_person_create_indices(ahnen_id, person) +
                  newline + self.link_person_to_parents(ahnen_id, person) +
                  newline + self.print_life_stats(ahnen_id, person, persons)
                 )
        return result


    def visit_generation(self, gen_num: int, generation: List[Tuple[int, 'Person']], families: Dict[str, Family]) -> str:
        """
          An entire generation, loops over the persons in the generation
        """
        if skip_first_gen and gen_num == 0:
            return ""
        latex = newline + newline
        latex += LaTeX.section(tr.t('Generation') + " " + str(gen_num + 1))
        latex += newline + newline
        for ahnen_id, person in generation:
            latex += self.visit_person(ahnen_id, person)
        return latex


# 📕 LaTeX and PDF generation

In [ ]:
#@title LaTeX pedigree generator and file creation. Calls the visitor class for processing
def group_by_generation(ahnen_map: Dict[int, Person]) -> Dict[int, Dict[str, List[Any]]]:
    """
      Helper function to prepare the generations
    """
    gen_groups = defaultdict(lambda: {'persons': [], 'families': []})
    for person_id, person in ahnen_map.items():
        gen = calculate_generation(person_id)
        gen_groups[gen]['persons'].append(person)
    return dict(sorted(gen_groups.items(), key=lambda x: x[0]))


def generate_documentation(generations: Dict[int, List[Tuple[int, 'Person']]], families: Dict[str, Family]) -> str:
    """
      Entry point for LaTeX file generation
    """
    result = ""
    visitors: List[GenerationVisitor] = [LatexGenerationVisitor()]
    for gen_num, generation in generations.items():
      for visitor in visitors:
            result += visitor.visit_generation(gen_num, generation, families)
    return result

In [ ]:
#@title Write latex file. The provided structure is minimal, you will want to adapt this
#%%skip
import subprocess
import os

# After generating
tex_content = generate_documentation(generations, families)
tex_path = 'pedigree.tex'
pdf_path = 'pedigree.pdf'


# Write full document (add preamble!)
full_tex = r"""
\documentclass[a4paper,10pt,openany]{book}
\usepackage[a4paper, margin=2.5cm]{geometry} %needed for luatex
\usepackage{emptypage}
\usepackage{fancybox}
\usepackage{afterpage}
\usepackage{bclogo}
"""

if (lang == 'en'):
  full_tex += "\\usepackage[english]{babel}"
elif (lang == 'nl'):
  full_tex += "\\usepackage[dutch]{babel}"

full_tex += r"""
%\usepackage[dutch]{babel}
\usepackage{wrapfig}
\usepackage{hyperref}
\usepackage{caption}
\usepackage{subcaption}
\usepackage{fontspec}

\usepackage{ebgaramond}
\setmainfont{Ebgaramond}

\usepackage{makeidx}
\usepackage{float}
\usepackage{tabularx}

\raggedbottom
\makeindex


\begin{document}
\tableofcontents
\mainmatter
\chapter{""" + tr.t('Pedigree') + r"""}
""" + tex_content + r"""
\printindex
\backmatter
\end{document}
"""
with open(tex_path, 'w', encoding='utf-8') as f:
    f.write(full_tex)



In [ ]:
#@title Call lualatex on file. The resulting PDF will be downloaded afterwards
#%%skip
!apt-get update -qq
!apt-get install -y texlive-latex-base texlive-latex-recommended \
    texlive-latex-extra texlive-fonts-recommended texlive-fonts-extra \
    texlive-plain-generic dvipng texlive-luatex texlive-xetex \
    texlive-binaries texlive-extra-utils texlive-lang-european  \
    texlive-lang-european texlive-pstricks texlive-pictures \
    fonts-ebgaramond -y
!tlmgr install babel-dutch wrapfig
!kpsewhich bclogo.sty
!texhash


print ("We will see one lualatex pass, followed by makeindex, followed by two more lualatex passes. This is a long process depending on the size of the initial GEDCOM file, please be patient")
!lualatex --shell-escape -interaction=nonstopmode -file-line-error pedigree.tex  > /dev/null 2>&1
print (ok + "First pass")
!makeindex pedigree > /dev/null 2>&1
print (ok + "Makeindex")
!lualatex --shell-escape -interaction=nonstopmode -file-line-error pedigree.tex  > /dev/null 2>&1
print (ok + "Second pass")
!lualatex --shell-escape -interaction=nonstopmode -file-line-error pedigree.tex  > /dev/null 2>&1
print (ok + "Third pass")

print(ok + "Done")

!ls -la pedigree.pdf  # CONFIRM file exists + size
!pwd                 # Must be /content/

# Only works if drive is mounted
from google.colab import files
import time

print("Triggering download...")
time.sleep(1)
files.download('pedigree.pdf')


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
dvipng is already the newest version (1.15-1.1).
fonts-ebgaramond is already the newest version (0.016+git20210310.42d4f9f2-1).
texlive-extra-utils is already the newest version (2021.20220204-1).
texlive-fonts-extra is already the newest version (2021.20220204-1).
texlive-fonts-recommended is already the newest version (2021.20220204-1).
texlive-lang-european is already the newest version (2021.20220204-1).
texlive-latex-base is already the newest version (2021.20220204-1).
texlive-latex-extra is already the newest version (2021.20220204-1).
texlive-latex-recommended is already the newest version (2021.20220204-1).
texlive-luatex is already the newest version (2021.20220204-1).
texlive-pictures is alread

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>